In [9]:
import requests
import json
import pandas as pd
import datetime as dt

def get_binance_bars(symbol, interval, startTime, endTime):
    url = "https://api.binance.com/api/v1/klines"
    
    # Konvertieren der Start- und Endzeiten in Millisekunden
    startTime = str(int(startTime.timestamp() * 1000))
    endTime = str(int(endTime.timestamp() * 1000))
    limit = '10000'
    
    # Erstellen der Anfrageparameter
    req_params = {'symbol': symbol, 'interval': interval, 'startTime': startTime, 'endTime': endTime, 'limit': limit}
    
    # Senden der Anfrage an die Binance API und Laden der Antwort in einen DataFrame
    df = pd.DataFrame(json.loads(requests.get(url, params=req_params).text))
    
    # Überprüfen, ob der DataFrame leer ist
    if df.empty:
        return None
    
    # Verarbeiten der Daten und Umbenennen der Spalten
    df = df.iloc[:, 0:6]
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    
    # Konvertieren der Spalten in die richtigen Datentypen
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    
    # Konvertieren der 'datetime' Spalte von Millisekunden seit Epoch zu einem lesbaren Datum
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    
    # Hinzufügen einer Spalte für den angepassten Schlusskurs
    df['adj_close'] = df['close']
    
    return df

# Beispiel für die Verwendung der Funktion
if __name__ == "__main__":
    symbol = 'BTCUSDT'
    interval = '15m'
    startTime = dt.datetime(2024, 1, 1)
    endTime = dt.datetime(2024, 5, 1)
    
    df = get_binance_bars(symbol, interval, startTime, endTime)
    if df is not None:
        print(df.head())
        df.to_csv('bitcoin_data_15m_2024.csv', index=False)
    else:
        print("Keine Daten gefunden.")

             datetime      open      high       low     close     volume  \
0 2024-01-01 08:00:00  42492.46  42529.91  42452.58  42514.28  137.17234   
1 2024-01-01 08:15:00  42514.28  42537.53  42477.60  42482.00   93.47006   
2 2024-01-01 08:30:00  42482.00  42504.79  42461.10  42492.91   89.52769   
3 2024-01-01 08:45:00  42492.91  42556.50  42489.95  42549.99  107.02356   
4 2024-01-01 09:00:00  42550.00  42577.88  42537.18  42574.24  151.36603   

   adj_close  
0   42514.28  
1   42482.00  
2   42492.91  
3   42549.99  
4   42574.24  
